In [152]:
import pandas as pd
import sklearn
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from numpy import reshape
from sklearn import metrics
from collections import Counter 

In [153]:
data = pd.read_csv('train_dec04_task2.csv')

In [154]:
# split train and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)

In [155]:
# split features and labels
X_train = train_data.iloc[:, :7]
y_train = train_data.iloc[:, 7]

X_test = test_data.iloc[:, :7]
y_test = test_data.iloc[:, 7]

pred_data = pd.read_csv('test_dec04_task2_only_features.csv')

In [156]:
# check the imbalance of the data
Counter(y_train)

Counter({'C': 1209, 'B': 1600, 'A': 791})

In [157]:
# use SMOTE to balance the data
# deprecated: SMOTE + TomekLinks is not better than SMOTE
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
X_res, y_res = SMOTE(random_state=42).fit_resample(X_train, y_train)
# X_res, y_res = TomekLinks().fit_resample(X_re, y_re)

# check the imbalance of the data again
print(f"Training target statistics: {Counter(y_res)}")
print(f"Testing target statistics: {Counter(y_test)}")

Training target statistics: Counter({'C': 1600, 'B': 1600, 'A': 1600})
Testing target statistics: Counter({'B': 400, 'C': 291, 'A': 209})


In [158]:
# tranform the labels to numbers to fit XGBoost
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

y_res = le.fit_transform(y_res)

In [159]:
# build XGBClassifier model
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.36)

# train data with train_data
# xgboostModel.fit(X_train, y_train)
xgboostModel.fit(X_res, y_res)

# predict with the test_data
predicted = xgboostModel.predict(X_test)

In [160]:
# print the accuracy
# print('訓練集: ',xgboostModel.score(X_train,y_train))
print('訓練集: ',xgboostModel.score(X_res,y_res))
print('測試集: ',xgboostModel.score(X_test,y_test))

訓練集:  1.0
測試集:  0.81


In [161]:
# predict the real given data
answer = xgboostModel.predict(pred_data)
answer = le.inverse_transform(answer)

In [162]:
da = {"Id": [x for x in range(1, len(answer)+1)], "Category": answer}
df = pd.DataFrame (da)

In [163]:
df.to_csv("output.csv", index=False)